# 言語処理100本ノック 2020 (Rev 2)



## 第6章: 機械学習

### 50. データの入手・整形

In [59]:
import pandas as pd

df_org = pd.read_csv('NewsAggregatorDataset/newsCorpora.csv',sep='\t', header=None, names=['id', 'title', 'url', 'publisher','category','story','hostname','timestamp'])

df_org

,id,title,url,publisher,category,story,hostname,timestamp
0,1,"Fed official says weak data caused by weather,...",http://www.latimes.com/business/money/la-fi-mo...,Los Angeles Times,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.latimes.com,1394470370698
1,2,Fed's Charles Plosser sees high bar for change...,http://www.livemint.com/Politics/H2EvwJSK2VE6O...,Livemint,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.livemint.com,1394470371207
2,3,US open: Stocks fall after Fed official hints ...,http://www.ifamagazine.com/news/us-open-stocks...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371550
3,4,"Fed risks falling 'behind the curve', Charles ...",http://www.ifamagazine.com/news/fed-risks-fall...,IFA Magazine,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.ifamagazine.com,1394470371793
4,5,Fed's Plosser: Nasty Weather Has Curbed Job Gr...,http://www.moneynews.com/Economy/federal-reser...,Moneynews,b,ddUyU0VZz0BRneMioxUPQVP6sIxvM,www.moneynews.com,1394470372027
...,...,...,...,...,...,...,...,...
422414,422933,Surgeons to remove 4-year-old's rib to rebuild...,http://www.cbs3springfield.com/story/26378648/...,WSHM-TV,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.cbs3springfield.com,1409229190251
422415,422934,Boy to have surgery on esophagus after battery...,http://www.wlwt.com/news/boy-to-have-surgery-o...,WLWT Cincinnati,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wlwt.com,1409229190508
422416,422935,Child who swallowed battery to have reconstruc...,http://www.newsnet5.com/news/local-news/child-...,NewsNet5.com,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.newsnet5.com,1409229190771
422417,422936,Phoenix boy undergoes surgery to repair throat...,http://www.wfsb.com/story/26368078/phoenix-boy...,WFSB,m,dpcLMoJD69UYMXMxaoEFnWql9YjQM,www.wfsb.com,1409229191071


In [60]:
# 抽出

publishers = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]

df = df_org.query("publisher in @publishers")

In [69]:
# シャッフル

df = df.sample(frac=1)

df.head()

,id,title,url,publisher,category,story,hostname,timestamp
147773,148109,Home > Tom Hanks > Tom Hanks And Steven Spielb...,http://www.contactmusic.com/story/tom-hanks-an...,Contactmusic.com,e,d6WbEbbyA67h6sM7cFFa77tmNGOFM,www.contactmusic.com,1398170093312
91913,91989,Twitter morphs further into Facebook with filt...,http://www.dailymail.co.uk/sciencetech/article...,Daily Mail,t,dhzjRpgru5-m7GMn0BusZ54_VSSXM,www.dailymail.co.uk,1396990234953
99311,99508,Rodrigo Santoro Opens Up About The Price He Pa...,http://www.huffingtonpost.com/2014/04/09/rodri...,Huffington Post,e,dyjwYV6L2OQ1JEMFhOWDVwX4jc-vM,www.huffingtonpost.com,1397242472058
6217,6218,Shailene Woodley arrives at LAX with hunky The...,http://www.dailymail.co.uk/tvshowbiz/article-2...,Daily Mail,e,ddfEq1wKc7iCzGMvYSy_ekodamTeM,www.dailymail.co.uk,1394598701413
273189,273635,Saturn's Northern Lights Glow Luminous Blue In...,http://www.huffingtonpost.com/2014/06/03/satur...,Huffington Post,t,dxrfehFJtNT5_4MyclRysvJ2xqIUM,www.huffingtonpost.com,1401893313949


In [70]:
# 振り分け
train_rows = round(len(df) * 0.8)
valid_rows = round(len(df) * 0.1)
train_df = df[0:train_rows]
valid_df = df[train_rows:train_rows+valid_rows]
test_df =  df[train_rows+valid_rows:]
print(len(df),len(train_df),len(valid_df),len(test_df),len(train_df)+len(valid_df)+len(test_df))


13340 10672 1334 1334 13340


In [71]:
# ファイル出力

#train_df[["id","category","title","url","publisher"]].to_csv('train.txt',sep='\t',index=False,header=False)
train_df[["category","title"]].to_csv('train.txt',sep='\t',index=False,header=False)
valid_df[["category","title"]].to_csv('valid.txt',sep='\t',index=False,header=False)
test_df[["category","title"]].to_csv('test.txt',sep='\t',index=False,header=False)

In [75]:
# 事例数確認

#train_df["category"].unique()
#train_df.query("title.str.contains('The Best Reactions')",engine='python')[["id","title","url"]]
df_org.query("title.str.contains('The Best Reactions')",engine='python')[["id","title","url"]]

#!cat train.txt | cut -f 1 | sort | uniq -c
#!cat valid.txt | cut -f 1 | sort | uniq -c
#!cat test.txt | cut -f 1 | sort | uniq -c


,id,title,url
110292,110489,SPOILER ALERT: The Best Reactions To Joffrey's...,http://theurbandaily.com/2014/04/14/best-react...
114825,115161,SPOILER ALERT: The Best Reactions To Joffrey's...,http://michronicleonline.com/2014/04/14/spoile...
210193,210629,The Best Reactions to the Supposed Video of So...,http://time.com/96572/the-best-reactions-to-th...
210277,210713,The Best Reactions To The Supposed Video of So...,http://www.contactmusic.com/article/gotham-rid...
